In [ ]:
import ee
import geemap
import utils

utils.ee_init()

In [ ]:
start_date = ee.Date('2015-06-27')
end_date = ee.Date('2025-01-01')
aoi = ee.FeatureCollection(
    'projects/ee-yangluhao990714/assets/downstream_tiny_aoi'
).geometry()
aoi_grid = ee.FeatureCollection('projects/ee-yangluhao990714/assets/downstream_grid')
m = geemap.Map(height='800px')
m.centerObject(ee_object=aoi, zoom=10)
index = 0
for aoi_grid_feature in aoi_grid.getInfo()['features']:
    if index is not 80:
        index += 1
        continue
    aoi = ee.Feature(aoi_grid_feature['geometry']).geometry()
    sentinel_2_l2a = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterBounds(aoi) \
        .filterDate(start_date, end_date)
    sentinel_2_l2a = sentinel_2_l2a.map(lambda img: img.clip(aoi))
    sentinel_2_l2a = sentinel_2_l2a.remove_clouds()
    sentinel_2_l2a = sentinel_2_l2a.band_rename()
    # sentinel_2_l2a: ee.ImageCollection = sentinel_2_l2a.temporal_composite(start_date, end_date, 'monthly')
    sentinel_2_l2a = sentinel_2_l2a.map(lambda img: img.nbr())
    sentinel_2_l2a = sentinel_2_l2a.map(lambda img: img.ndvi())
    sentinel_2_l2a = sentinel_2_l2a.map(lambda img: img.ndsi())
    sentinel_2_l2a = sentinel_2_l2a.map(lambda img: img.ndwi())
    ccdc_input = sentinel_2_l2a.map(
        lambda img: img.updateMask(img.select('NDSI').lt(0).And(img.select('NDWI').lt(0)))
    )
    ccdc_input = ccdc_input.select(['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2', 'NBR', 'NDVI'])
    ccdc_input = ccdc_input.map(lambda img: img.clip(aoi))
    ccdc_result: ee.Image = ee.Algorithms.TemporalSegmentation.Ccdc(
        ccdc_input,
        minObservations=12,
        dateFormat=2,
        chiSquareProbability=0.99,
        maxIterations=25000,
    )
    bands_list = ee.List([
        ee.Dictionary({'tBreak': [
            'tBreak_0', 'tBreak_1', 'tBreak_2', 'tBreak_3', 'tBreak_4',
            'tBreak_5', 'tBreak_6', 'tBreak_7', 'tBreak_8', 'tBreak_9'
        ]}),
        ee.Dictionary({'changeProb': [
            'changeProb_0', 'changeProb_1', 'changeProb_2', 'changeProb_3', 'changeProb_4',
            'changeProb_5', 'changeProb_6', 'changeProb_7', 'changeProb_8', 'changeProb_9'
        ]}),
    ])
    # keys() and values() will return ee.List with length 1, so we need to get(0) to get the actual parameters
    ccdc_result_flat_list = bands_list.map(
        lambda band: ccdc_result.select([ee.Dictionary(band).keys().get(0)]).arrayPad([10], 0) \
            .arrayFlatten([ee.Dictionary(band).values().get(0)])
    )
    ccdc_result_flat = ee.Image(ccdc_result_flat_list.get(0)).addBands(ee.Image(ccdc_result_flat_list.get(1)))
    file_name = f'ccdc_result_{index}'
    geemap.ee_export_image_to_asset(
        image=ccdc_result_flat,
        description='export_' + file_name,
        assetId=f'projects/ee-yangluhao990714/assets/ccdc_result_millis/{file_name}',
        scale=10,
        region=aoi,
        maxPixels=1e13,
        crs='EPSG:4326',
    )
    index += 1